In [ ]:
# Import libraries
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
import os
import shutil
import random
import cv2

from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, losses

In [ ]:
# Connect to Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Read all the images from the folder
input_folder = "/content/drive/MyDrive/Images"

x_train_noisy = []
x_train_denoised = []
x_test_noisy = []
x_test_denoised = []

for i, filename in enumerate(os.listdir(input_folder)):

    if filename.endswith(".jpg") or filename.endswith(".png"):
        # Determine if this is a noised or denoised image
        is_denoised = filename.endswith("b.jpg") or filename.endswith("b.png")
        if is_denoised:
            continue

        denoised_filename = filename[:-4] + "b.jpg" if filename.endswith(".jpg") else filename[:-4] + "b.png"

        img_noisy = cv2.imread(os.path.join(input_folder, filename))
        img_denoised = cv2.imread(os.path.join(input_folder, denoised_filename))
        
        # Assign the image to the train or test set with 80:20 ratio
        if random.random() < 0.8:
            x_train_noisy.append(np.array(img_noisy))
            x_train_denoised.append(np.array(img_denoised))
        else:
            x_test_noisy.append(np.array(img_noisy))
            x_test_denoised.append(np.array(img_denoised))

x_train_noisy = np.array(x_train_noisy)
x_train_denoised = np.array(x_train_denoised)
x_test_noisy = np.array(x_test_noisy)
x_test_denoised = np.array(x_test_denoised)

In [ ]:
# Covert pixels values from 0...255 to 0...1
x_train_noisy = x_train_noisy.astype('float32') / 255.
x_train_denoised = x_train_denoised.astype('float32') / 255.
x_test_noisy = x_test_noisy.astype('float32') / 255.
x_test_denoised = x_test_denoised.astype('float32') / 255.

In [ ]:
# Display the number of images in train and test set
len(x_train_noisy), len(x_train_denoised), len(x_test_noisy), len(x_test_denoised)

(3393, 3393, 859, 859)

In [ ]:
# Define U-Net structure for image denoising

IMG_WIDTH = IMG_HEIGHT = 256
IMG_CHANNELS = 3

inputs = tf.keras.layers.Input((IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS)) # (256, 256, 3)

# Contraction path
c1 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(inputs) # (256, 256, 16)
c1 = tf.keras.layers.Dropout(0.1)(c1)
c1 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c1)
p1 = tf.keras.layers.MaxPooling2D((2, 2))(c1) # (128, 128, 16)

c2 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p1) # (128, 128, 32)
c2 = tf.keras.layers.Dropout(0.1)(c2)
c2 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c2)
p2 = tf.keras.layers.MaxPooling2D((2, 2))(c2) # (64, 64, 32)
 
c3 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p2) # (64, 64, 64)
c3 = tf.keras.layers.Dropout(0.2)(c3)
c3 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c3)
p3 = tf.keras.layers.MaxPooling2D((2, 2))(c3) # (32, 32, 64)
 
c4 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p3) # (32, 32, 128)
c4 = tf.keras.layers.Dropout(0.2)(c4)
c4 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c4)
p4 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(c4) # (16, 16, 128)
 
c5 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p4) # (16, 16, 256)
c5 = tf.keras.layers.Dropout(0.3)(c5)
c5 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c5)

# Expansive path 
u6 = tf.keras.layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c5) # (32, 32, 128)
u6 = tf.keras.layers.concatenate([u6, c4]) # (32, 32, 256)
c6 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u6) # (32, 32, 128)
c6 = tf.keras.layers.Dropout(0.2)(c6)
c6 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c6)
 
u7 = tf.keras.layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c6) # (64, 64, 64)
u7 = tf.keras.layers.concatenate([u7, c3])
c7 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u7)
c7 = tf.keras.layers.Dropout(0.2)(c7)
c7 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c7)
 
u8 = tf.keras.layers.Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(c7) # (128, 128, 32)
u8 = tf.keras.layers.concatenate([u8, c2])
c8 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u8)
c8 = tf.keras.layers.Dropout(0.1)(c8)
c8 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c8)
 
u9 = tf.keras.layers.Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same')(c8) # (256, 256, 16)
u9 = tf.keras.layers.concatenate([u9, c1], axis=3)
c9 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u9)
c9 = tf.keras.layers.Dropout(0.1)(c9)
c9 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c9) 
 
outputs = tf.keras.layers.Conv2D(3, (1, 1), activation='sigmoid')(c9) # (256, 256, 3)

In [ ]:
# Training
model = tf.keras.Model(inputs=[inputs], outputs=[outputs])
model.compile(optimizer='adam', loss=losses.MeanSquaredError())
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)
model.fit(x_train_noisy, x_train_denoised, # we force our network to generate a denoised image from a noisy one
                epochs=100, # maximum number of epochs
                shuffle=True,
                validation_split=0.2, # train data is splitted into train and validation set in a ratio of 80:20
                callbacks=[callback]) # early stopping - prevention from overfitting

Epoch 1/100
85/85 [==============================] - 45s 446ms/step - loss: 0.0257 - val_loss: 0.0073
Epoch 2/100
85/85 [==============================] - 30s 355ms/step - loss: 0.0068 - val_loss: 0.0045
Epoch 3/100
85/85 [==============================] - 31s 360ms/step - loss: 0.0044 - val_loss: 0.0034
Epoch 4/100
85/85 [==============================] - 31s 363ms/step - loss: 0.0036 - val_loss: 0.0032
Epoch 5/100
85/85 [==============================] - 31s 365ms/step - loss: 0.0031 - val_loss: 0.0028
Epoch 6/100
85/85 [==============================] - 31s 366ms/step - loss: 0.0029 - val_loss: 0.0027
Epoch 7/100
85/85 [==============================] - 31s 365ms/step - loss: 0.0028 - val_loss: 0.0026
Epoch 8/100
85/85 [==============================] - 31s 366ms/step - loss: 0.0026 - val_loss: 0.0026
Epoch 9/100
85/85 [==============================] - 31s 366ms/step - loss: 0.0026 - val_loss: 0.0025
Epoch 10/100
85/85 [==============================] - 31s 366ms/step - loss: 0.002

In [ ]:
# Prediction on new images
denoised_imgs = model.predict(x_test_noisy)

27/27 [==============================] - 4s 148ms/step


In [ ]:
# Visualize the results for 20 different random images
n = 20
m = len(x_test_noisy)
random_indices = random.sample(range(m), n)

for i in random_indices:
    plt.figure()

    # display original + noise
    ax = plt.subplot(1, 3, 1)
    plt.title("Noisy")
    plt.imshow(cv2.cvtColor(x_test_noisy[i], cv2.COLOR_BGR2RGB))
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # display reconstruction
    bx = plt.subplot(1, 3, 2)
    plt.title("Reconstructed")
    plt.imshow(cv2.cvtColor(denoised_imgs[i], cv2.COLOR_BGR2RGB))
    bx.get_xaxis().set_visible(False)
    bx.get_yaxis().set_visible(False)

    # display original denoised
    bx = plt.subplot(1, 3, 3)
    plt.title("Original denoised")
    plt.imshow(cv2.cvtColor(x_test_denoised[i], cv2.COLOR_BGR2RGB))
    bx.get_xaxis().set_visible(False)
    bx.get_yaxis().set_visible(False)

    plt.show()

Output hidden; open in https://colab.research.google.com to view.